In [1]:
from IPython.display import HTML
from IPython.display import display

tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide the GUI code, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

############### GUI CODE BELOW ##################
from SeismicReduction import *
import ipywidgets as widgets
import pickle
import matplotlib
from IPython.display import clear_output
from ipywidgets import interact, VBox, HBox
set_seed(42)

# Layout template widget
lint = inline_start = widgets.IntText(
    value=1300,
    description='Start:',
    width=0.05,
    layout=widgets.Layout(height='auto', width='auto')
)

# title
title_text = widgets.HTML(
    value="<h1><b>SeismicReduction GUI : </b></h1> <br> \
           <h1>Unsupervised Machine Learning on Seismic Data</h>",
)

# Explanation text widgets
explain_load = widgets.HTML(
    value="1. Data Loading - <b>Run First</b> <br>" \
        ">Files names can be absolute or relative to the directory this notebook is run from.<br>" \
        ">Only necessary to re-run data loading if using a new data set.",
)

explain_processing = widgets.HTML(
    value="2. Processing - <b>Run Second</b> <br>" \
        "> A new data input will be processed with every run.<br>" \
        "> <b>Note:</b> If flattening is set to false, the seismic data will instead be cropped.<br>" \
        "The window will use <i>above_add<i> to <i>below_add<i> as the top and bottom data index's"
)

explain_model = widgets.HTML(
    value="3. Model analysis <br>" \
        "> Models will be run on the last input processed in previous step.<br>" \
        "> Multiple models can be run on this input, no need to regenerate inputs.<br>" \
        "> 2D Umap projections can be run and rerun on a trained model, no need to re-run models.<br>" 
)

explain_visualisation = widgets.HTML(
    value="4. Visualisation <br>" \
        "> Visualisation is run on the last model analysis.<br>" \
        "> The plot attribute automatically updates when the toggle value below is updated.<br>" \
)

# Data loading widgets
data_files_title = widgets.HTML(
    value="<b>File Pathnames:</b>",
)

near_text = widgets.Text(description='Near SEGY:', value='./data/3d_nearstack.sgy', layout=lint.layout)
far_text = widgets.Text(description='Far SEGY:', value='./data/3d_farstack.sgy', layout=lint.layout)
horizon_text = widgets.Text(description='Horizon .txt:', value='./data/Top_Heimdal_subset.txt', layout=lint.layout)

inline_range_title = widgets.HTML(
    value="<b>In-line range:<b>",
)

xline_range_title = widgets.HTML(
    value="<b>X-line range:<b>",
)

inline_start = widgets.IntText(
    value=1300,
    description='Start:',
    width=0.05,
    layout=lint.layout
)

inline_stop = widgets.IntText(
    value=1502,
    description='Stop:',
    layout=lint.layout
)

inline_step = widgets.IntText(
    value=2,
    description='Step:',
    layout=lint.layout
)

xline_start = widgets.IntText(
    value=1500,
    description='Start:',
    layout=lint.layout
    
)


xline_stop = widgets.IntText(
    value=2002,
    description='Stop:',
    layout=lint.layout
)

xline_step = widgets.IntText(
    value=2,
    description='Step:',
    layout=lint.layout
)

load_button = widgets.Button(
    description='Load Data', 
    layout=lint.layout)

# Data processing:
flattening_title = widgets.HTML(
    value="<b>Horizon Flattening:<b>",
    layout=lint.layout
)

norm_title = widgets.HTML(
    value="<b>Normalisation:<b>",
    layout=lint.layout
)

flat_option = widgets.Dropdown(
    options=[True, False],
    value=True,
    description='True/False:',
    disabled=False,
    layout=lint.layout
)

above_add = widgets.IntText(
    value=12,
    description='Above Add:', 
    layout=lint.layout)

below_add = widgets.IntText(
    value=52,
    description='Below Add:', 
    layout=lint.layout)

norm_option = widgets.Dropdown(
    options=[True, False],
    value=True,
    description='True/False:',
    disabled=False,
)

process_button = widgets.Button(
    description='Process Input', 
    layout=lint.layout)

# to_2d update function
twod_title = widgets.HTML(
    value="<b>2-D embedding (UMAP) parameters:<b>",
)

twod_neighbours = widgets.IntText(
    value=50,
    description='Neighbours:', 
    layout=lint.layout
)

twod_dist = widgets.BoundedFloatText(
    value=1e-2,
    min=1e-5,
    max=1,
    step=5e-3,
    description='Min Dist:', 
    layout=lint.layout
)

twod_button = widgets.Button(
    description='Update 2D embedding', 
    layout=lint.layout
)

# PCA
pca_title = widgets.HTML(
    value="<b>PCA:<b>",
)

pca_components = widgets.IntText(
    value=2,
    description='Components:', 
    layout=lint.layout)

pca_button = widgets.Button(
    description='Run Pca', 
    layout=lint.layout)

# UMAP
umap_title = widgets.HTML(
    value="<b>UMAP:<b>",
)

umap_neighbours = widgets.IntText(
    value=50,
    description='Neighbours:', 
    layout=lint.layout)

umap_dist = widgets.BoundedFloatText(
    value=1e-2,
    min=0.0001,
    max=1,
    step=0.0001,
    description='Min Dist:', 
    layout=lint.layout)

umap_button = widgets.Button(
    description='Initialise Umap',
    layout=lint.layout)

# VAE
vae_title = widgets.HTML(
    value="<b>VAE parameters:<b>",
)

epoch_num = widgets.IntText(
    value=10,
    min=3,
    description='Epochs:', 
    layout=lint.layout)

latent_dim = widgets.IntText(
    value=4,
    description='Latent size:', 
    layout=lint.layout)

learn_rate = widgets.BoundedFloatText(
    value=1e-2,
    min=0.0001,
    max=1,
    step=0.005,
    description='Learn Rate:', 
    layout=lint.layout)

vae_button = widgets.Button(
    description='Run Vae', 
    layout=lint.layout)

# BVAE
bvae_title = widgets.HTML(
    value="<b>beta-VAE parameters:<b>",
)

bvae_beta_value = widgets.IntText(
    value=5,
    step=0.5,
    description='beta value:', 
    layout=lint.layout)

bvae_button = widgets.Button(
    description='Run beta-Vae', 
    layout=lint.layout)

# Plotting:
plot_title = widgets.HTML(
    value="<b>Plot parameters:<b>",
)

# drop down widget
attr_dropdown = widgets.Dropdown(
    options=['horizon', 'FF'],
    description='Plot Attribute:',
)

# Declare output widget
PLOToutput = widgets.Output(layout={'border': '4px solid black'})

GENERALoutput = widgets.Output(layout={'border': '4px solid red'})

clear_button = widgets.Button(description='Clear Output')

## LOGIC

# Data load:
@load_button.on_click
def load_on_click(b):
    ### Client loader
    with GENERALoutput:
        clear_output()
        print('Loading SEGY: \n')

        dataholder = DataHolder("Data", [inline_start.value, inline_stop.value, inline_step.value],
                                [xline_start.value, xline_stop.value, xline_step.value])
        dataholder.add_near(near_text.value);
        dataholder.add_far(far_text.value);
        dataholder.add_horizon(horizon_text.value)
        dataholder.add_well('well_1', 36, 276 // 2)

        # save to binary file
        if not os.path.exists('./pickled/'):
            os.mkdir('./pickled/')
        
        with open("./pickled/data.pickle", "wb") as file_:
            pickle.dump(dataholder, file_, -1)
        file_.close()
        print('\nData Loaded successfully!\n')


# Data process:
@process_button.on_click
def process_on_click(b):
    # load data
    file_pi2 = open('./pickled/data.pickle', 'rb')
    dataholder = pickle.load(file_pi2)
    file_pi2.close()

    # processing
    with GENERALoutput:
        clear_output()
        processor = Processor(dataholder)
        if flat_option.value == True: # FLATTEN
            input1 = processor(flatten=[True, above_add.value, below_add.value], normalise=norm_option.value)
        elif flat_option.value == False: # CROP
            input1 = processor(crop=[True, above_add.value, below_add.value], normalise=norm_option.value)

        # save to binary file
        with open("./pickled/input.pickle", "wb") as file_:
            pickle.dump(input1, file_, -1)
        file_.close()
        print('Data Input processed successfully')

# Run Pca
@pca_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()

    with PLOToutput:
        clear_output()

    # RUN
    with GENERALoutput:
        clear_output()
        pca = PcaModel(input1)
        pca.reduce(pca_components.value)
        pca.to_2d(umap_neighbours=twod_neighbours.value, umap_dist=twod_dist.value)

    # First plot
    makeplot(pca, attr_dropdown.value)

    # save to binary file
    with open("./pickled/model.pickle", "wb") as file_:
        pickle.dump(pca, file_, -1)
    file_.close()

# Run Umap
@umap_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()

    with PLOToutput:
        clear_output()

    # UMAP RUN
    with GENERALoutput:
        clear_output()
        umap = UmapModel(input1)
        umap.reduce()

    # First plot
    with GENERALoutput:
        print('\nPlease use the update button to run plain UMAP on input\n')

    # save to binary file
    with open("./pickled/model.pickle", "wb") as file_:
        pickle.dump(umap, file_, -1)
    file_.close()


# Run VAE
@vae_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()

    with PLOToutput:
        clear_output()

    with GENERALoutput:
        % matplotlib inline
        # VAE RUN
        vae = VaeModel(input1)
        vae.reduce(epochs=epoch_num.value, hidden_size=latent_dim.value, lr=learn_rate.value)
        vae.to_2d(umap_neighbours=twod_neighbours.value, umap_dist=twod_dist.value)



        # save to binary file
        with open("./pickled/model.pickle", "wb") as file_:
            pickle.dump(vae, file_, -1)
        file_.close()
    
    % matplotlib notebook
    makeplot(vae, attr_dropdown.value)
    
# Run bVAE
@bvae_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/input.pickle', 'rb')
    input1 = pickle.load(file_pi2)
    file_pi2.close()

    with PLOToutput:
        clear_output()

    with GENERALoutput:
        % matplotlib inline
        # VAE RUN
        bvae = BVaeModel(input1)
        bvae.reduce(epochs=epoch_num.value, hidden_size=latent_dim.value, lr=learn_rate.value,
                     beta=bvae_beta_value.value)
        bvae.to_2d(umap_neighbours=twod_neighbours.value, umap_dist=twod_dist.value)

        # save to binary file
        with open("./pickled/model.pickle", "wb") as file_:
            pickle.dump(bvae, file_, -1)
        file_.close()
    
    % matplotlib notebook
    makeplot(bvae, attr_dropdown.value)
    

# update 2d UMAP
@twod_button.on_click
def run_on_click(b):
    # load input
    file_pi2 = open('./pickled/model.pickle', 'rb')
    model = pickle.load(file_pi2)
    file_pi2.close()

    with PLOToutput:
        clear_output()

    # RUN
    with GENERALoutput:
        clear_output()
        model.to_2d(umap_neighbours=twod_neighbours.value, umap_dist=twod_dist.value, verbose=True)

    # First plot
    makeplot(model, attr_dropdown.value)

    # save to binary file
    with open("./pickled/model.pickle", "wb") as file_:
        pickle.dump(model, file_, -1)
    file_.close()

@attr_dropdown.observe
def on_change(change):
    # load embedding
    file2 = open('./pickled/model.pickle', 'rb')
    model = pickle.load(file2)
    file2.close()

    #  if dropdown change
    if change['type'] == 'change' and change['name'] == 'value':
        #         print (change['new'])
        makeplot(model, change['new'])


def makeplot(model, attr):
    % matplotlib notebook
    with PLOToutput:
        clear_output()

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.set(xlabel='Latent Variable 1', ylabel='Latent Variable 2',
               title='Model used: {}, Trace Attribute: {}'.format(model.name, attr),
               aspect='equal')
        s = ax.scatter(model.two_dimensions[:, 0], model.two_dimensions[:, 1], s=0.5, c=model.attributes[attr])
        c = plt.colorbar(s, shrink=0.7, orientation='vertical')
        c.set_label(label=attr, rotation=90, labelpad=10)


@clear_button.on_click
def clear_it(arg):
    with GENERALoutput:
        clear_output()
    with PLOToutput:
        clear_output()

## GUI LAYOUT
%matplotlib notebook

# Load tab
tab1 = VBox(children=[explain_load,
                      HBox(children=[VBox(children=[data_files_title, near_text, far_text, horizon_text]),
                                     VBox(children=[inline_range_title, inline_start, inline_stop, inline_step]),
                                     VBox(children=[xline_range_title, xline_start, xline_stop, xline_step]),
                                     ], layout=widgets.Layout(height='150px')),
                      load_button
                      ])

# processing tab
tab2 = VBox(children=[explain_processing,
                      HBox(children=[VBox(children=[flattening_title, flat_option, above_add, below_add, ]),
                                     VBox(children=[norm_title, norm_option], layout=widgets.Layout(height='150px'))
                                     ]),
                      process_button
                      ])

# model tab
accordion = widgets.Accordion(children=[HBox(children=[
                                            VBox(children=[pca_title, pca_components, pca_button]), 
                                            VBox(children=[twod_title, twod_neighbours, twod_dist, twod_button])]),
                                        HBox(children=[
                                            VBox(children=[umap_title, umap_button]),
                                            VBox(children=[twod_title, twod_neighbours, twod_dist, twod_button])]),
                                        HBox(children=[
                                            VBox(children=[vae_title, epoch_num, learn_rate, latent_dim, vae_button]),
                                            VBox(children=[twod_title, twod_neighbours, twod_dist, twod_button])]),
                                        HBox(children=[
                                            VBox(children=[bvae_title, epoch_num, learn_rate, latent_dim, bvae_beta_value, bvae_button]),
                                            VBox(children=[twod_title, twod_neighbours, twod_dist, twod_button])]),
                                       ]
                             )
accordion.set_title(0, 'PCA - Principal Component Analysis')
accordion.set_title(1, 'UMAP - Universal Manifold Aproximation Projection')
accordion.set_title(2, 'VAE - Variational Auto-Encoder')
accordion.set_title(3, 'β-VAE - β-Varational Auto-Encoder')

tab3 = VBox(children=[explain_model,
                      HBox(children=[VBox(children=[]),
                                     VBox(children=[]),
                                     ]),
                      accordion
                      ])

# visualisation
tab4 = VBox(children=[explain_visualisation,
                      HBox(children=[VBox(children=[plot_title, attr_dropdown]),
                                     VBox(children=[]),
                                     ])
                      ])

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, '1. Data Loading')
tab.set_title(1, '2. Data Processing')
tab.set_title(2, '3. Model Selection')
tab.set_title(3, '4. Visualisation')

VBox(children=[title_text, tab, clear_button, PLOToutput, GENERALoutput])
